In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd

/home/jw/anaconda3/envs/kobert/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
data = pd.read_csv('./data.csv')
# print(data)
data

,문장,class1(input),class2(output),class3(model),s_n
0,제가 찍은 캠핑장 사진에서는 어떤 캠핑용품들과 야생동물들이 있나요?,image,Object_detection,Yolo,1
1,"이 영화 속에서 등장하는 의상과 악세서리들을 구분하고, 각각이 어디에 위치해 있는지...",image,Object_detection,Yolo,1
2,"제가 찍은 가을 풍경 사진에서는 어떤 나무와 잎들이 있는지 구분하고, 각각이 어디에...",image,Object_detection,Yolo,1
3,"이 영상에서 등장하는 인물들과 사물들을 구분하고, 어디에 위치해 있는지 알려주세요.",image,Object_detection,Yolo,1
4,"제가 찍은 해외 거리 사진에서는 어떤 사람들과 건물들이 있는지 구분하고, 각각이 어...",image,Object_detection,Yolo,1
...,...,...,...,...,...
1494,내가 쓰고 있는 책에 대한 몇 가지 독특한 삽화를 만들어야 해. 그런데 어떻게 시작...,text,generation,dall-e,3
1495,나는 많은 커스텀 비주얼을 생성해야 하는 프로젝트를 진행하고 있어. 하지만 직접 모...,text,generation,dall-e,3
1496,프리젠테이션용 그래픽을 만들려고 해. 하지만 디자인에 소질이 없어. 이미지 생성 알...,text,generation,dall-e,3
1497,독특한 비주얼이 많이 필요한 게임을 만들고 있어. 하지만 나는 디자인 쪽 전문가가 ...,text,generation,dall-e,3


In [35]:
data['class3(model)'].unique()

array(['Yolo', 'efficientNet', 'Unet', 'gan', 'dall-e'], dtype=object)

In [3]:
data.loc[(data['class3(model)'] == "Yolo"), 'class3(model)'] = 0  #공포 => 0
data.loc[(data['class3(model)'] == "efficientNet"), 'class3(model)'] = 1  #놀람 => 1
data.loc[(data['class3(model)'] == "Unet"), 'class3(model)'] = 2  #놀람 => 1
data.loc[(data['class3(model)'] == "gan"), 'class3(model)'] = 3  #놀람 => 1
data.loc[(data['class3(model)'] == "dall-e"), 'class3(model)'] = 4  #놀람 => 1

In [4]:
# data = data.dropna(subset = ['문장'], how = 'any',  axis=0)
data = data.dropna()
data.tail(10)

,문장,class1(input),class2(output),class3(model),s_n
1489,시각화해야 할 데이터가 많아. 그런데 어떻게 하면 시각적으로 매력적으로 보이게 할 ...,text,generation,4,3
1490,나는 새 제품을 만들고 있고 마케팅 자료용 비주얼을 만들어야 해. 하지만 나는 숙련...,text,generation,4,3
1491,가상 현실 프로젝트를 진행 중이야. 현실적인 환경을 만들어야 하는데 직접 만들 리소...,text,generation,4,3
1492,프레젠테이션을 위해 독특한 시각 자료를 만들어야 해. 하지만 나는 전문가가 아니야....,text,generation,4,3
1493,나는 연구 프로젝트를 진행 중이야. 맞춤 시각 자료를 만들어야 하는데 직접 만들 수...,text,generation,4,3
1494,내가 쓰고 있는 책에 대한 몇 가지 독특한 삽화를 만들어야 해. 그런데 어떻게 시작...,text,generation,4,3
1495,나는 많은 커스텀 비주얼을 생성해야 하는 프로젝트를 진행하고 있어. 하지만 직접 모...,text,generation,4,3
1496,프리젠테이션용 그래픽을 만들려고 해. 하지만 디자인에 소질이 없어. 이미지 생성 알...,text,generation,4,3
1497,독특한 비주얼이 많이 필요한 게임을 만들고 있어. 하지만 나는 디자인 쪽 전문가가 ...,text,generation,4,3
1498,마케팅 자료를 위한 맞춤형 그래픽을 만들고 싶어. 하지만 직접 할 수 있는 디자인 ...,text,generation,4,3


In [5]:
data['class3(model)'].unique()

array([0, 1, 2, 3, 4], dtype=object)

In [6]:
data_list = []

for q, label in zip(data['문장'], data['class3(model)']):
 
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

data_list

[['제가 찍은 캠핑장 사진에서는 어떤 캠핑용품들과 야생동물들이 있나요?', '0'],
 ['이 영화 속에서 등장하는 의상과 악세서리들을 구분하고, 각각이 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 가을 풍경 사진에서는 어떤 나무와 잎들이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 영상에서 등장하는 인물들과 사물들을 구분하고, 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 해외 거리 사진에서는 어떤 사람들과 건물들이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 노래의 뮤직비디오에서 나오는 출연진들을 구분하고, 각각이 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 산속 사진에서는 어떤 동물과 식물들이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 영화 속에서 등장하는 자동차와 오토바이를 구분하고, 각각이 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 도시 사진에서는 어떤 건물과 인물들이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 드라마에서 등장하는 배경과 인물들을 구분하고, 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 해변 사진에서는 어떤 파도와 모래사장, 구름이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 소설에서 등장하는 도시와 건물들을 구분하고, 어디에 위치해 있는지 알려주세요.', '0'],
 ['제가 찍은 공원 사진에서는 어떤 나무와 꽃들이 있는지 구분하고, 각각이 어디에 위치해 있는지도 알려주세요.', '0'],
 ['이 영화 속에서 등장하는 도시들과 건물들을 구분하고, 어디에 위치해 있는지 알려주세요.', '0'],
 ['내가 찍은 여름 바다 사진에서 물고기가 어디에 있는지 알려줘.', '0'],
 ['이 사진에서 자동차 모델을 분류하고 그 차량이 어디쯤 위치하

In [7]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0)

In [8]:
df = pd.DataFrame(dataset_train, columns=['sentence', 'label'])
df['label'].value_counts()

2    284
4    270
0    268
1    266
3    261
Name: label, dtype: int64

In [9]:
import numpy as np
from torch.utils.data import Dataset, DataLoader

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/jw/workspace/nlp_modelname_classification/.cache/kobert_v1.zip
using cached model. /home/jw/workspace/nlp_modelname_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/jw/workspace/nlp_modelname_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [71]:
data_test[0]

(array([   2, 3434, 6896, 1023, 7022, 5195, 7088, 4610, 3860, 2585, 4257,
        6896,  517,   46, 1562, 6018, 7307, 6983,  833, 3939, 1746, 5940,
        3223, 6896, 3563, 7794, 7318, 3169, 7303,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(32, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [74]:
train_dataloader

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/home/jw/anaconda3/envs/kobert/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  5%|▍         | 1/22 [00:00<00:20,  1.01it/s]

epoch 1 batch id 1 loss 1.690744400024414 train acc 0.21875


100%|██████████| 22/22 [00:06<00:00,  3.54it/s]


epoch 1 train acc 0.2294034090909091


100%|██████████| 3/3 [00:00<00:00,  8.72it/s]


...

epoch 20 batch id 1 loss 0.002625425346195698 train acc 1.0


100%|██████████| 22/22 [00:05<00:00,  3.93it/s]


epoch 20 train acc 1.0


100%|██████████| 3/3 [00:00<00:00,  9.05it/s]

epoch 20 test acc 0.9895833333333334


In [16]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("yolo")
            elif np.argmax(logits) == 1:
                test_eval.append("efficientnet")
            elif np.argmax(logits) == 2:
                test_eval.append("Unet")
            elif np.argmax(logits) == 3:
                test_eval.append("gan")
            elif np.argmax(logits) == 4:
                test_eval.append("dall-e")

        print(f">> {test_eval[0]} 모델을 사용하세요.")

using cached model. /home/jw/workspace/nlp_modelname_classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [21]:
predict('텍스트를 입력했을때 이미지로 변환해주는 모델은 뭐야?')

>> dall-e 모델을 사용하세요.
